In [577]:
import pandas as pd
from configurations import basic_parameters as bp

In [578]:
csv_dataframe = bp['dataframe_dir'] + '/' + bp['dataframe_file']

In [579]:
names = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']

df = pd.read_csv(csv_dataframe, names=names, usecols=['Open time', 'Open', 'High', 'Low', 'Close'])

In [580]:
def computeRSI (data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

In [581]:
def stochastic(data, k_window, d_window, window):
    
    # input to function is one column from df
    # containing closing price or whatever value we want to extract K and D from
    
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()
    
    stoch = ( (data - min_val) / (max_val - min_val) ) * 100
    
    K = stoch.rolling(window=k_window, center=False).mean() 
    #K = stoch
    
    D = K.rolling(window=d_window, center=False).mean() 


    return K, D

In [582]:
df['RSI'] = computeRSI(df['Close'], 14)
df['K'], df['D'] = stochastic(df['RSI'], 3, 3, 14)

In [583]:
df['time UTC'] = pd.to_datetime(df['Open time'], unit='ms', origin='unix')
df.dropna(inplace=True)
df

,Open time,Open,High,Low,Close,RSI,K,D,time UTC
31,1677715231000,23633.57,23634.82,23632.96,23634.12,60.781924,70.630393,56.267568,2023-03-02 00:00:31
32,1677715232000,23633.59,23634.14,23632.32,23633.13,56.039172,63.861023,64.376390,2023-03-02 00:00:32
33,1677715233000,23632.28,23633.18,23632.00,23633.18,56.224954,38.564245,57.685220,2023-03-02 00:00:33
34,1677715234000,23632.96,23632.96,23632.02,23632.05,50.981232,15.458772,39.294680,2023-03-02 00:00:34
35,1677715235000,23632.05,23633.49,23631.80,23633.49,56.543297,22.415855,25.479624,2023-03-02 00:00:35
...,...,...,...,...,...,...,...,...,...
86331,1677801595000,23465.54,23465.64,23464.80,23465.45,54.189816,61.018658,53.889356,2023-03-02 23:59:55
86332,1677801596000,23465.43,23465.46,23465.03,23465.39,53.553541,75.687165,64.562074,2023-03-02 23:59:56
86333,1677801597000,23465.04,23466.34,23465.04,23466.34,61.301373,79.728506,72.144776,2023-03-02 23:59:57
86334,1677801598000,23466.23,23466.23,23465.65,23465.65,54.226058,69.225085,74.880252,2023-03-02 23:59:58


In [584]:
df.loc[(df['D'] == 0) & (df['RSI'] < 25), 'buy?'] = True
df.loc[df['buy?'] == True]

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?
796,1677715996000,23645.01,23645.54,23641.92,23642.25,18.927554,0.0,0.0,2023-03-02 00:13:16,True
797,1677715997000,23642.08,23642.08,23638.27,23641.55,18.107212,0.0,0.0,2023-03-02 00:13:17,True
798,1677715998000,23641.53,23641.56,23640.52,23641.00,17.466653,0.0,0.0,2023-03-02 00:13:18,True
799,1677715999000,23640.53,23640.99,23639.62,23640.85,17.287038,0.0,0.0,2023-03-02 00:13:19,True
1565,1677716766000,23696.32,23696.84,23694.25,23695.53,24.795097,0.0,0.0,2023-03-02 00:26:06,True
...,...,...,...,...,...,...,...,...,...,...
84355,1677799617000,23451.08,23451.73,23450.80,23450.95,18.270097,0.0,0.0,2023-03-02 23:26:57,True
84356,1677799618000,23451.32,23451.32,23449.00,23450.14,16.584965,0.0,0.0,2023-03-02 23:26:58,True
84687,1677799950000,23457.10,23457.10,23454.33,23454.92,20.839077,0.0,0.0,2023-03-02 23:32:30,True
84845,1677800108000,23434.54,23434.54,23430.32,23431.23,14.831080,0.0,0.0,2023-03-02 23:35:08,True


In [585]:
rolling_seconds = 60

df['min_val'] = df['Low'].shift(-rolling_seconds).rolling(window=rolling_seconds).min()
df['max_val'] = df['High'].shift(-rolling_seconds).rolling(window=rolling_seconds).max()

df['max_val_%'] = (df['max_val'] - df['Open'].shift(-1)) * 100 / df['Open'].shift(-1)
df['min_val_%'] = (df['min_val'] - df['Open'].shift(-1)) * 100 / df['Open'].shift(-1)

#df.dropna(inplace=True)
#df[:50]

In [586]:
tpsl = 0.075

In [587]:
sl = df.loc[(df['buy?'] == True) & (df['min_val_%'] < -tpsl) & (df['max_val_%'] < tpsl)]
sl

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?,min_val,max_val,max_val_%,min_val_%
2895,1677718096000,23653.98,23654.32,23652.64,23653.68,23.874357,0.0,0.0,2023-03-02 00:48:16,True,23636.00,23656.26,0.008328,-0.077322
9779,1677724980000,23489.41,23489.99,23486.84,23487.00,24.149335,0.0,0.0,2023-03-02 02:43:00,True,23449.50,23488.08,0.002810,-0.161448
22274,1677737484000,23423.16,23424.27,23422.90,23423.70,23.816487,0.0,0.0,2023-03-02 06:11:24,True,23401.79,23425.00,0.005507,-0.093580
39163,1677754388000,23357.53,23357.84,23356.19,23356.61,24.346311,0.0,0.0,2023-03-02 10:53:08,True,23337.37,23359.30,0.012373,-0.081519
45723,1677760953000,23371.60,23372.00,23368.00,23368.02,21.962662,0.0,0.0,2023-03-02 12:42:33,True,23319.00,23371.94,0.016818,-0.209731
45724,1677760954000,23368.01,23369.94,23367.53,23368.01,21.951243,0.0,0.0,2023-03-02 12:42:34,True,23319.00,23371.94,0.012709,-0.213831
45748,1677760978000,23347.44,23348.68,23345.79,23345.96,10.295298,0.0,0.0,2023-03-02 12:42:58,True,23319.00,23348.59,0.006510,-0.120229
45749,1677760979000,23347.07,23347.07,23344.50,23345.81,10.209354,0.0,0.0,2023-03-02 12:42:59,True,23319.00,23348.59,0.011908,-0.114839
48809,1677764042000,23377.81,23378.78,23373.70,23374.45,24.848800,0.0,0.0,2023-03-02 13:34:02,True,23343.05,23384.44,0.042397,-0.134677
48810,1677764043000,23374.53,23375.00,23370.02,23372.23,23.246189,0.0,0.0,2023-03-02 13:34:03,True,23343.05,23384.44,0.049245,-0.127840


In [588]:
tp = df.loc[(df['buy?'] == True) & (df['max_val_%'] > tpsl) & (df['min_val_%'] > -tpsl)]
tp

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?,min_val,max_val,max_val_%,min_val_%
1565,1677716766000,23696.32,23696.84,23694.25,23695.53,24.795097,0.0,0.0,2023-03-02 00:26:06,True,23695.23,23715.64,0.084530,-0.001604
5334,1677720535000,23540.84,23541.00,23535.01,23537.00,17.362140,0.0,0.0,2023-03-02 01:28:55,True,23531.87,23563.48,0.112589,-0.021711
5335,1677720536000,23536.98,23537.00,23533.19,23535.30,15.945014,0.0,0.0,2023-03-02 01:28:56,True,23531.87,23563.48,0.119778,-0.014531
5336,1677720537000,23535.29,23536.19,23531.87,23535.26,15.912104,0.0,0.0,2023-03-02 01:28:57,True,23533.32,23563.48,0.124032,-0.004122
12669,1677727872000,23482.60,23482.80,23482.32,23482.56,23.277164,0.0,0.0,2023-03-02 03:31:12,True,23482.25,23503.21,0.087938,-0.001320
26664,1677741878000,23347.69,23348.33,23341.00,23345.37,15.405209,0.0,0.0,2023-03-02 07:24:38,True,23341.13,23369.20,0.105935,-0.014307
26665,1677741879000,23344.47,23350.98,23341.13,23344.31,14.752645,0.0,0.0,2023-03-02 07:24:39,True,23342.42,23369.20,0.101862,-0.012850
31750,1677746968000,23444.58,23444.58,23441.03,23442.90,22.944235,0.0,0.0,2023-03-02 08:49:28,True,23440.00,23465.37,0.096191,-0.012029
31751,1677746969000,23442.82,23443.34,23440.00,23440.10,19.106057,0.0,0.0,2023-03-02 08:49:29,True,23440.57,23465.37,0.105757,-0.000043
36631,1677751853000,23364.00,23366.29,23360.52,23364.01,19.835622,0.0,0.0,2023-03-02 10:10:53,True,23360.00,23385.43,0.091894,-0.016949


In [589]:
tp.shape[0]/sl.shape[0]

1.588235294117647

In [591]:
tp['max_val_%'].sum()+sl['min_val_%'].sum()

0.5313044452100546